In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cobra
from cobra.io import read_sbml_model, write_sbml_model
from cobra.flux_analysis import flux_variability_analysis
from tqdm import tqdm

In [ ]:
class Predation:
    def add_dynamic_bounds(self, model, y, biomass):
        """Use external concentrations to bound the uptake flux of glucose. \n
        :param sbml model: The model you want to add dynamic bounds. \n
        :param list(int) y: A list of the metabolite quantities in environment. In order: glucose, amonium.
        :param int biomass: Quantities of Biomass in the environment.
        """
        glucose, oxygen, ammonium = y  # expand the boundary species (Can put more)

        glucose_max_import = (
            -10 * glucose / (5 + glucose)
        )  # calculate the max import value for the metabolites
        ammonium_max_import = -10 * ammonium / (5 + ammonium)
        biomass_max_import = -10 * biomass / (5 + biomass)

        model.reactions.EX_nh4_e.lower_bound = (
            ammonium_max_import  # change the limit of importation with the max import
        )

        self.prey.reactions.EX_glc__D_e.lower_bound = glucose_max_import

        self.pred.reactions.EX_glu_L_e.bounds = [0, 0]
        self.pred.reactions.EX_Biomass_e.lower_bound = biomass_max_import

    def update_metabolites(fluxes, y, biomass):
        """Updtate the metabolites quantities depending of the fluxes. \n
        :param list(int) fluxes: The list of the fluxes. \n
        :param list(int) y: A list of the metabolite quantities in environment. In order: glucose, oxygen, amonium, isoleucine. \n
        :param int biomass: Quantities of Biomass in the environment.
        """
        glucose, oxygen, ammonium = y  # expand the boundary species (Can put more)

        glucose = glucose + fluxes["EX_glc__D_e"]  # calculate the number of metabolites
        oxygen = oxygen + fluxes["EX_o2_e"]
        ammonium = ammonium + fluxes["EX_nh4_e"]
        if "BIOMASS_Ec_iML1515_core_75p37M" in fluxes:
            biomass = biomass + fluxes["BIOMASS_Ec_iML1515_core_75p37M"]
        if "EX_Biomass_e" in fluxes:
            biomass = biomass + fluxes["EX_Biomass_e"]

        if glucose < 0:
            glucose = (
                0  # if the value goes below zero set the number of metaboltie to zero
            )
        if oxygen < 0:
            oxygen = 0
        if ammonium < 0:
            ammonium = 0
        if biomass < 0:
            biomass = 0

        y = (glucose, oxygen, ammonium)  # changing the value of the metabolites
        return y, biomass

    def add_predation(self):
        """Add the predation reaction to the predator model."""
        ...

    def run(self):
        """Run the flux balance analysis with predation between the prey and the predator."""
        self.fluxes_prey = []
        self.fluxes_preda = []
        self.time_point = [0]
        it = 0

        biomass = 0  # quantity of biomass in environment
        self.Biomass_Pred = []
        self.Biomass_Prey = []

        with tqdm(total=self.tf) as pbar:
            while t < self.tf:
                self.add_dynamic_bounds(self.prey, y, biomass)  ## For Escherichia coli
                FBA_1 = self.prey.optimize()  # perform flux balance analysis (FBA)
                fluxes_1 = FBA_1.fluxes.get(
                    self.reactions_list_prey
                )  # store the fluxes of the FBA
                fluxes_1 *= 0.1  # multiply by Biomass Concentration to get flux
                self.fluxes_prey.append(fluxes_1)  # store all the fluxes
                y, biomass = self.update_metabolites(fluxes_1, y, biomass)
                self.Biomass_Prey.append(biomass)

                self.add_dynamic_bounds(
                    self.preda, y, biomass
                )  ## For Myxococcus xanthus
                FBA_2 = self.pred.optimize()
                fluxes_2 = FBA_2.fluxes.get(self.reactions_list_preda)
                fluxes_2 *= 0.1
                self.fluxes_preda.append(fluxes_2)
                y, biomass = self.update_metabolites(fluxes_2, y, biomass)
                self.Biomass_Pred.append(
                    self.Biomass_Pred[it - 1] + fluxes_2["OF_BIOMASS"]
                )

                self.time_point.append(self.time_point[it - 1] + self.steps)
                it += 1
                t += self.steps
                pbar.update(self.steps)

    def plot_flux(self):
        """Plot the fluxes of biomass of the prey against the predator"""
        self.run()  # Run the predator interaction

        biomass_E = []
        biomass_M = []

        for i in self.fluxes_preda:
            self.fluxes_preda.append(i["OF_BIOMASS"])  # taking the biomass flux
        for j in self.fluxes_prey:
            self.fluxes_prey.append(j["BIOMASS_Ec_iML1515_core_75p37M"])

        ax = plt.subplot(111)
        ax.plot(
            self.time_point[1:], biomass_E, color="blue", label="Prey"
        )  # plot the biomass flux of E_coli
        ax2 = plt.twinx(ax)
        ax2.plot(self.time_point[1:], biomass_M, color="red", label="Preda")

        ax.set_ylabel("Biomass", color="blue")  # set the name
        ax2.set_ylabel("Biomass", color="red")
        ax.set_xlabel("Time")

        ax.set_title(
            "Fluxes of the biomass from E.coli against M.xanthus"
        )  # set the title
        ax.legend(bbox_to_anchor=(1.15, 1), loc="upper left")  # show the legend
        ax2.legend(bbox_to_anchor=(1.15, 0.90), loc="upper left")

    def plot_biomass(self):
        """Plot the fluxes of biomass of the prey against the predator"""
        self.run()  # Run the predator interaction

        ax = plt.subplot(111)
        ax.plot(
            self.time_point[1:], self.Biomass_Prey[1:], color="blue", label="Prey"
        )  # plot the biomass flux of E_coli
        ax2 = plt.twinx(ax)
        ax2.plot(
            self.time_point[1:], self.Biomass_Pred[1:], color="red", label="Predator"
        )

        ax.set_ylabel("Biomass", color="blue")  # set the name
        ax2.set_ylabel("Biomass", color="red")
        ax.set_xlabel("Time")

        ax.set_title(
            "Fluxes of the biomass from E.coli against M.xanthus"
        )  # set the title
        ax.legend(bbox_to_anchor=(1.15, 1), loc="upper left")  # show the legend
        ax2.legend(bbox_to_anchor=(1.15, 0.90), loc="upper left")

    def __init__(
        self, prey, pred, metabolite: list, tf: float = 50, dt: float = 0.5
    ) -> None:
        self.prey = prey
        self.pred = pred
        self.metabolite = metabolite
        self.tf = tf
        self.steps = dt

        self.reactions_list_prey = []  # get all reaction of E_coli model
        for reaction in self.prey.reactions:
            self.reactions_list_prey.append(reaction.id)

        self.reactions_list_preda = []
        for reaction in self.preda.reactions:
            self.reactions_list_preda.append(reaction.id)